In [5]:
import tensorflow as tf
from tensorflow import keras

# !pip install tensorflow-datasets
import tensorflow_datasets as tfds

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(f"Tensorflow version {tf.__version__}")

Tensorflow version 2.13.0


In [7]:
# Download project data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv -nv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv -nv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

2024-01-06 15:06:48 URL:https://cdn.freecodecamp.org/project-data/sms/train-data.tsv [358233/358233] -> "train-data.tsv.1" [1]
2024-01-06 15:06:49 URL:https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv [118774/118774] -> "valid-data.tsv.1" [1]


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):



  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
